In [39]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import matplotlib.pyplot as plt

import tensorflow as tf
tf.compat.v1.enable_v2_behavior()

from tf_agents.agents.ddpg import critic_network
from tf_agents.agents.sac import sac_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import actor_distribution_network
from tf_agents.networks import normal_projection_network
from tf_agents.policies import greedy_policy
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

import gym_train_env

In [40]:
env_name = "train-tf-v1"

# use "num_iterations = 1e6" for better results,
num_iterations = 100000

initial_collect_steps = 1000 
collect_steps_per_iteration = 1 
replay_buffer_capacity = 1000000 

batch_size = 256

critic_learning_rate = 3e-4 
actor_learning_rate = 3e-4 
alpha_learning_rate = 3e-4 
target_update_tau = 0.005 
target_update_period = 1
gamma = 0.99 
reward_scale_factor = 1.0 
gradient_clipping = None 

actor_fc_layer_params = (256, 256)
critic_joint_fc_layer_params = (256, 256)

log_interval = 5 

num_eval_episodes = 30
eval_interval = 100

In [41]:
env = suite_gym.load(env_name)
env.reset()

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([1.  , 0.03, 0.  , 0.  , 0.03, 1.  , 0.03, 0.02, 0.  , 0.03, 1.  ,
       0.01, 0.  , 0.02, 0.01, 1.  , 0.  , 0.25, 0.  , 0.  , 0.25, 0.  ,
       0.3 , 0.4 , 0.  , 0.3 , 0.  , 0.8 , 0.  , 0.4 , 0.8 , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.1 ]))

In [42]:
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

In [43]:
observation_spec = train_env.observation_spec()
action_spec = train_env.action_spec()
critic_net = critic_network.CriticNetwork(
    (observation_spec, action_spec),
    observation_fc_layer_params=None,
    action_fc_layer_params=None,
    joint_fc_layer_params=critic_joint_fc_layer_params)

In [44]:
def normal_projection_net(action_spec,init_means_output_factor=0.1):
  return normal_projection_network.NormalProjectionNetwork(
      action_spec,
      mean_transform=None,
      state_dependent_std=True,
      init_means_output_factor=init_means_output_factor,
      std_transform=sac_agent.std_clip_transform,
      scale_distribution=True)

In [45]:
actor_net = actor_distribution_network.ActorDistributionNetwork(
    observation_spec,
    action_spec,
    fc_layer_params=actor_fc_layer_params,
    continuous_projection_net=normal_projection_net)

In [46]:
global_step = tf.compat.v1.train.get_or_create_global_step()
tf_agent = sac_agent.SacAgent(
    train_env.time_step_spec(),
    action_spec,
    actor_network=actor_net,
    critic_network=critic_net,
    actor_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=actor_learning_rate),
    critic_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=critic_learning_rate),
    alpha_optimizer=tf.compat.v1.train.AdamOptimizer(
        learning_rate=alpha_learning_rate),
    target_update_tau=target_update_tau,
    target_update_period=target_update_period,
    td_errors_loss_fn=tf.compat.v1.losses.mean_squared_error,
    gamma=gamma,
    reward_scale_factor=reward_scale_factor,
    gradient_clipping=gradient_clipping,
    train_step_counter=global_step)
tf_agent.initialize()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [47]:
eval_policy = greedy_policy.GreedyPolicy(tf_agent.policy)
collect_policy = tf_agent.collect_policy

In [48]:
def compute_avg_return(environment, policy, num_episodes=5):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


compute_avg_return(eval_env, eval_policy, num_eval_episodes)

0.0

In [49]:
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=tf_agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

In [50]:
initial_collect_driver = dynamic_step_driver.DynamicStepDriver(
        train_env,
        collect_policy,
        observers=[replay_buffer.add_batch],
        num_steps=initial_collect_steps)
initial_collect_driver.run()

(TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([2], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 51), dtype=float64, numpy=
 array([[1.        , 0.03      , 0.        , 0.        , 0.03      ,
         1.        , 0.03      , 0.02      , 0.        , 0.03      ,
         1.        , 0.01      , 0.        , 0.02      , 0.01      ,
         1.        , 0.        , 0.25      , 0.        , 0.        ,
         0.25      , 0.        , 0.3       , 0.4       , 0.        ,
         0.3       , 0.        , 0.8       , 0.        , 0.4       ,
         0.8       , 0.        , 0.19444444, 0.21736111, 0.23402778,
         0.16041667, 0.00277778, 0.04583333, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.      

In [51]:
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size, num_steps=2).prefetch(3)

iterator = iter(dataset)

In [52]:
collect_driver = dynamic_step_driver.DynamicStepDriver(
    train_env,
    collect_policy,
    observers=[replay_buffer.add_batch],
    num_steps=collect_steps_per_iteration)

In [ ]:
try:
  %%time
except:
  pass

# Optimize by wrapping some of the code in a graph using TF function.
tf_agent.train = common.function(tf_agent.train)
collect_driver.run = common.function(collect_driver.run)

# Reset the train step
tf_agent.train_step_counter.assign(0)

# Evaluate the agent's policy once before training.
avg_return = compute_avg_return(eval_env, eval_policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collect a few steps using collect_policy and save to the replay buffer.
  for _ in range(collect_steps_per_iteration):
    collect_driver.run()

  # Sample a batch of data from the buffer and update the agent's network.
  experience, unused_info = next(iterator)
  train_loss = tf_agent.train(experience)

  step = tf_agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, eval_policy, num_eval_episodes)
    print('step = {0}: Average Return = {1}'.format(step, avg_return))
    eval_py_env.render()
    print('')
    returns.append(avg_return)

step = 5: loss = 472.3922424316406
step = 10: loss = 329.2236633300781
step = 15: loss = 255.52243041992188
step = 20: loss = 255.2845458984375
step = 25: loss = 266.1014099121094
step = 30: loss = 264.69012451171875
step = 35: loss = 270.8209533691406
step = 40: loss = 256.78607177734375
step = 45: loss = 252.53936767578125
step = 50: loss = 245.06507873535156
step = 55: loss = 238.94720458984375
step = 60: loss = 244.5365753173828
step = 65: loss = 248.63687133789062
step = 70: loss = 264.3808288574219
step = 75: loss = 269.29632568359375
step = 80: loss = 276.6889343261719
step = 85: loss = 277.90618896484375
step = 90: loss = 279.9167175292969
step = 95: loss = 300.8321533203125
step = 100: loss = 293.3328552246094
step = 100: Average Return = 0.0
 4:25 -  3:23 -  4:31 -  3: 4 -  5:32 -  5:16 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 105: loss = 284.28924560546875
step = 110: loss = 306.034912109375
step = 115: loss

step = 940: loss = 651.1392822265625
step = 945: loss = 635.4365844726562
step = 950: loss = 628.0150146484375
step = 955: loss = 659.94140625
step = 960: loss = 630.9212646484375
step = 965: loss = 641.5733642578125
step = 970: loss = 695.7386474609375
step = 975: loss = 707.7969970703125
step = 980: loss = 694.3424072265625
step = 985: loss = 652.544677734375
step = 990: loss = 591.896240234375
step = 995: loss = 582.3394775390625
step = 1000: loss = 605.63671875
step = 1000: Average Return = 0.0
 3:45 -  2:55 -  1:45 -  5:50 -  2:40 -  0:51 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 1005: loss = 608.0790405273438
step = 1010: loss = 583.5538940429688
step = 1015: loss = 574.7196044921875
step = 1020: loss = 552.1671142578125
step = 1025: loss = 627.3525390625
step = 1030: loss = 588.91162109375
step = 1035: loss = 604.9602661132812
step = 1040: loss = 616.6787109375
step = 1045: loss = 576.9129638671875
step = 1050: l

step = 1840: loss = -20.49597930908203
step = 1845: loss = -17.398788452148438
step = 1850: loss = -17.713293075561523
step = 1855: loss = -11.6035795211792
step = 1860: loss = -15.023723602294922
step = 1865: loss = -16.440242767333984
step = 1870: loss = -10.364795684814453
step = 1875: loss = -11.352108001708984
step = 1880: loss = -3.3308067321777344
step = 1885: loss = -15.49571418762207
step = 1890: loss = -21.210508346557617
step = 1895: loss = -15.828629493713379
step = 1900: loss = -9.496391296386719
step = 1900: Average Return = 0.0
 0:13 -  0:11 -  0: 6 -  0: 4 -  0:10 -  0:10 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 1905: loss = -8.18746566772461
step = 1910: loss = 8.936347961425781
step = 1915: loss = -0.7707376480102539
step = 1920: loss = -22.05416488647461
step = 1925: loss = 7.429924964904785
step = 1930: loss = 13.1044282913208
step = 1935: loss = -14.935036659240723
step = 1940: loss = 12.4597930908

step = 2700: Average Return = 0.0
 0: 6 -  0: 6 -  0: 5 -  0: 8 -  0: 8 -  0: 8 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 2705: loss = -26.064716339111328
step = 2710: loss = -26.250362396240234
step = 2715: loss = -22.63054656982422
step = 2720: loss = -24.274372100830078
step = 2725: loss = -21.971303939819336
step = 2730: loss = -23.242660522460938
step = 2735: loss = -22.266937255859375
step = 2740: loss = -20.78853988647461
step = 2745: loss = -24.063283920288086
step = 2750: loss = -21.28596305847168
step = 2755: loss = -25.021053314208984
step = 2760: loss = -23.479923248291016
step = 2765: loss = -24.727235794067383
step = 2770: loss = -20.35879898071289
step = 2775: loss = -24.975799560546875
step = 2780: loss = -24.562692642211914
step = 2785: loss = -27.056167602539062
step = 2790: loss = -25.73701286315918
step = 2795: loss = -24.577556610107422
step = 2800: loss = -27.78976821899414
step = 2800: Average Ret

step = 3545: loss = -27.459712982177734
step = 3550: loss = -22.901996612548828
step = 3555: loss = -27.70500946044922
step = 3560: loss = -25.658435821533203
step = 3565: loss = -28.125730514526367
step = 3570: loss = -25.829429626464844
step = 3575: loss = -24.843521118164062
step = 3580: loss = -29.597179412841797
step = 3585: loss = -28.084732055664062
step = 3590: loss = -9.680445671081543
step = 3595: loss = -26.80937385559082
step = 3600: loss = -25.810041427612305
step = 3600: Average Return = 1.0
 0: 6 -  0: 7 -  0: 4 -  0: 5 -  0: 6 -  0: 7 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 3605: loss = -29.56603240966797
step = 3610: loss = -27.66031265258789
step = 3615: loss = -27.92970085144043
step = 3620: loss = -26.371383666992188
step = 3625: loss = -28.233675003051758
step = 3630: loss = -27.069744110107422
step = 3635: loss = -27.50440216064453
step = 3640: loss = -27.505413055419922
step = 3645: loss = -28.4

...

step = 5240: loss = -36.93840789794922
step = 5245: loss = -38.09552764892578
step = 5250: loss = -39.1258544921875
step = 5255: loss = -27.22848892211914
step = 5260: loss = -38.387569427490234
step = 5265: loss = -37.75454330444336
step = 5270: loss = -36.988014221191406
step = 5275: loss = -37.75649642944336
step = 5280: loss = -37.97779083251953
step = 5285: loss = -38.347957611083984
step = 5290: loss = -37.88997268676758
step = 5295: loss = -37.3929443359375
step = 5300: loss = -37.812095642089844
step = 5300: Average Return = 1.0
 0: 3 -  0: 7 -  0: 5 -  0:14 -  0: 7 -  0: 7 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 5305: loss = -38.22441864013672
step = 5310: loss = -35.39101028442383
step = 5315: loss = -36.42237091064453
step = 5320: loss = -18.63270378112793
step = 5325: loss = -38.611915588378906
step = 5330: loss = -36.9307746887207
step = 5335: loss = -35.213035583496094
step = 5340: loss = -38.80307769775

step = 6100: Average Return = 1.0
 0: 3 -  0: 8 -  0: 6 -  0:10 -  0: 9 -  0: 9 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 6105: loss = -25.262264251708984
step = 6110: loss = -41.723960876464844
step = 6115: loss = -41.348514556884766
step = 6120: loss = -42.10622787475586
step = 6125: loss = -44.51729965209961
step = 6130: loss = -44.88883972167969
step = 6135: loss = -43.10123062133789
step = 6140: loss = -46.297279357910156
step = 6145: loss = -42.97117233276367
step = 6150: loss = -43.277122497558594
step = 6155: loss = -44.364349365234375
step = 6160: loss = -44.0761833190918
step = 6165: loss = -44.733367919921875
step = 6170: loss = -42.71491241455078
step = 6175: loss = -43.36640930175781
step = 6180: loss = -43.032684326171875
step = 6185: loss = -44.24937438964844
step = 6190: loss = -43.3284797668457
step = 6195: loss = -44.273677825927734
step = 6200: loss = -45.34071350097656
step = 6200: Average Return = 1

step = 6945: loss = -36.56512451171875
step = 6950: loss = -26.364408493041992
step = 6955: loss = -39.99595260620117
step = 6960: loss = -36.48961639404297
step = 6965: loss = -35.502445220947266
step = 6970: loss = -38.98858642578125
step = 6975: loss = -39.69908905029297
step = 6980: loss = -29.729829788208008
step = 6985: loss = -39.635955810546875
step = 6990: loss = -30.32673454284668
step = 6995: loss = -26.82474708557129
step = 7000: loss = -28.241119384765625
step = 7000: Average Return = 10.0
 0: 5 -  3:34 -  4:39 -  5:33 -  5:53 -  5:54 - 
 0:14 -  0:12 -  0: 8 -  0:13 -  0:17 -  0:17 - 
 0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 -  0: 0 - 

step = 7005: loss = -29.900217056274414
step = 7010: loss = -30.489105224609375
step = 7015: loss = -36.540870666503906
step = 7020: loss = -32.78793716430664
step = 7025: loss = -28.728425979614258
step = 7030: loss = -24.068822860717773
step = 7035: loss = -36.44414520263672
step = 7040: loss = -31.375911712646484
step = 7045: loss = -19.48

step = 7805: loss = 71.5701904296875
step = 7810: loss = 45.75348663330078
step = 7815: loss = 172.54541015625
step = 7820: loss = 164.9761962890625
step = 7825: loss = 65.12545776367188
step = 7830: loss = 77.27005004882812
step = 7835: loss = 175.35028076171875
step = 7840: loss = 178.077880859375
step = 7845: loss = 77.62174987792969
step = 7850: loss = 295.5406188964844
step = 7855: loss = 122.4742202758789
step = 7860: loss = 113.54364776611328
step = 7865: loss = 64.3555908203125
step = 7870: loss = 259.64215087890625
step = 7875: loss = 111.2965087890625
step = 7880: loss = 313.5599365234375
step = 7885: loss = 184.04714965820312
step = 7890: loss = 103.69575500488281
step = 7895: loss = 189.59805297851562
step = 7900: loss = 96.34716033935547
step = 7900: Average Return = 230.0
 0: 0 -  4:30 -  5:28 -  5:35 -  5:48 -  5:59 - 
 0: 0 -  4:27 -  5:26 -  5:31 -  5:46 -  5:57 - 
 0: 0 -  4:31 -  5:31 -  5:36 -  5:48 -  5:58 - 

step = 7905: loss = 183.26657104492188
step = 7910: los

step = 8695: loss = 1241.0748291015625
step = 8700: loss = 1358.6055908203125
step = 8700: Average Return = 238.0
 0: 0 -  4: 9 -  5: 2 -  5:30 -  5:57 -  5:59 - 
 0: 0 -  4: 0 -  4:57 -  5:28 -  5:57 -  5:59 - 
 0: 0 -  4: 2 -  4:59 -  5:29 -  5:57 -  5:59 - 

step = 8705: loss = 1078.7220458984375
step = 8710: loss = 1388.2723388671875
step = 8715: loss = 1260.788818359375
step = 8720: loss = 1258.1077880859375
step = 8725: loss = 1341.5950927734375
step = 8730: loss = 1269.9561767578125
step = 8735: loss = 1273.90869140625
step = 8740: loss = 1239.695556640625
step = 8745: loss = 1284.626220703125
step = 8750: loss = 1285.44287109375
step = 8755: loss = 1209.5830078125
step = 8760: loss = 1666.9178466796875
step = 8765: loss = 1303.3173828125
step = 8770: loss = 1284.683837890625
step = 8775: loss = 1425.5301513671875
step = 8780: loss = 1284.6234130859375
step = 8785: loss = 1214.570068359375
step = 8790: loss = 1306.1007080078125
step = 8795: loss = 1433.6278076171875
step = 8800:

In [19]:
train_checkpointer = common.Checkpointer(
    ckpt_dir='./temp2',
    max_to_keep=1,
    agent=tf_agent,
    policy=tf_agent.policy,
    replay_buffer=replay_buffer,
    global_step=global_step
)


In [20]:
train_checkpointer.save(global_step)

In [16]:
train_checkpointer.initialize_or_restore()

In [17]:
global_step = tf.compat.v1.train.get_global_step()

In [21]:
from simulation.simulator import Simulator
from simulation.utils import DemoNetwork1

sim = Simulator(DemoNetwork1)

tt = [[1,15,18,30,40,50],
     [1,15,18,30,40,50],
     [1,15,18,30,40,50]]

results = sim.evaluate_timetable(tt)
total_reward = results['passenger_increase']['reward_total'] + results['trains_delay']['reward_total'] + results['trains_increase']['reward_total']

In [22]:
total_reward

10.132577554090005

In [23]:
results

{'passenger_increase': {'1': 0.932765809589993,
  '5': 0.9848651623555311,
  '10': 0.8457620347559758,
  '25': 0.3661007667031763,
  '50': 0.18559928443649373,
  '100': 0.09357134090013501,
  'reward_total': 3.408664398741305},
 'trains_increase': {'0': 0.932765809589993,
  '1': 0.932765809589993,
  '2': 0.9534398888116747,
  '4': 0.9675121612230716,
  'reward_total': 3.786483669214732},
 'trains_delay': {'5': 0.9904375343973583,
  '10': 0.9950396164857847,
  '30': 0.6328884021512838,
  '60': 0.3190639330995414,
  'reward_total': 2.937429486133968},
 'timetables': [[1, 15, 18, 30, 40, 50],
  [1, 15, 18, 30, 40, 50],
  [1, 15, 18, 30, 40, 50]]}

In [27]:
from simulation.simulator import Simulator
from simulation.utils import DemoNetwork1

sim = Simulator(DemoNetwork1)

tt = [[0,1,2,3,4,5],
     [0,1,2,3,4,5],
     [0,1,2,3,4,5]]

results = sim.evaluate_timetable(tt)
total_reward = results['passenger_increase']['reward_total'] + results['trains_delay']['reward_total'] + results['trains_increase']['reward_total']

In [28]:
total_reward

9.526618475757955

In [29]:
results

{'passenger_increase': {'1': 0.9320708825573315,
  '5': 0.9846931755641167,
  '10': 0.8451960849590519,
  '25': 0.3656509153497105,
  '50': 0.18536270773378963,
  '100': 0.09350189600708014,
  'reward_total': 3.406475662171081},
 'trains_increase': {'0': 0.9320708825573315,
  '1': 0.9320708825573315,
  '2': 0.9454482279360668,
  '4': 0.9515288394718554,
  'reward_total': 3.761118832522585},
 'trains_delay': {'5': 0.9871697853604843,
  '10': 0.8747253478926693,
  '30': 0.33019062283136713,
  '60': 0.166938224979768,
  'reward_total': 2.359023981064289},
 'timetables': [[0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5], [0, 1, 2, 3, 4, 5]]}